In [13]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_digits, load_wine, fetch_covtype
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score

data = fetch_covtype()
X, y = data.data, data.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train - 1, dtype=torch.long)  # Adjust class labels for pytorch
y_test = torch.tensor(y_test - 1, dtype=torch.long)

class SimpleNN(nn.Module):
    def __init__(self, input_size, num_classes):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = SimpleNN(X_train.shape[1], len(np.unique(y_train)))
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

def train(model, X_train, y_train, X_test, y_test, epochs=20):
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = criterion(outputs, y_train)
        loss.backward()
        optimizer.step()

        model.eval()
        with torch.no_grad():
            test_outputs = model(X_test)
            test_loss = criterion(test_outputs, y_test).item()
            probs = torch.softmax(test_outputs, dim=1)
            _, test_preds = torch.max(test_outputs, 1)
            accuracy = accuracy_score(y_test, test_preds)
            auc = roc_auc_score(y_test.numpy(), probs.numpy(), multi_class='ovo')
        print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}, Test Loss: {test_loss:.4f}, Accuracy: {accuracy:.4f}, AUC: {auc:.4f}")

train(model, X_train, y_train, X_test, y_test)

def fgsm_attack(data, epsilon, data_grad):
    sign_data_grad = data_grad.sign()
    perturbed_data = data + epsilon * sign_data_grad
    return torch.clamp(perturbed_data, 0, 1)

def adversarial_test(model, X_test, y_test, epsilon_values=[0.05, 0.1, 0.2]):
    for epsilon in epsilon_values:
        correct = 0
        adv_probs = []
        model.eval()
        for i in range(len(X_test)):
            data, target = X_test[i], y_test[i]
            data.requires_grad = True

            output = model(data.unsqueeze(0))
            loss = criterion(output, target.unsqueeze(0))
            model.zero_grad()
            loss.backward()
            data_grad = data.grad.data
            perturbed_data = fgsm_attack(data, epsilon, data_grad)

            output = model(perturbed_data.unsqueeze(0))
            probs = torch.softmax(output, dim=1)
            adv_probs.append(probs.detach().numpy()[0])

            _, final_pred = output.max(1)
            correct += (final_pred == target).item()

        accuracy = correct / len(X_test)
        adv_probs = np.array(adv_probs)
        auc = roc_auc_score(y_test.numpy(), adv_probs, multi_class='ovo')
        print(f"Epsilon: {epsilon} - Adversarial Accuracy: {accuracy:.4f}, AUC: {auc:.4f}")

adversarial_test(model, X_test, y_test)

wine_data = load_wine()
X_wine, y_wine = wine_data.data, wine_data.target
X_wine_train, X_wine_test, y_wine_train, y_wine_test = train_test_split(X_wine, y_wine, test_size=0.3, random_state=42)

X_wine_train = scaler.fit_transform(X_wine_train)
X_wine_test = scaler.transform(X_wine_test)

X_wine_train = torch.tensor(X_wine_train, dtype=torch.float32)
X_wine_test = torch.tensor(X_wine_test, dtype=torch.float32)
y_wine_train = torch.tensor(y_wine_train, dtype=torch.long)
y_wine_test = torch.tensor(y_wine_test, dtype=torch.long)

model_wine = SimpleNN(X_wine_train.shape[1], len(np.unique(y_wine_train)))
optimizer = optim.Adam(model_wine.parameters(), lr=0.001)
train(model_wine, X_wine_train, y_wine_train, X_wine_test, y_wine_test)

adversarial_test(model_wine, X_wine_test, y_wine_test)


Epoch 1, Loss: 1.9375, Test Loss: 1.8988, Accuracy: 0.2366, AUC: 0.4998
Epoch 2, Loss: 1.8984, Test Loss: 1.8607, Accuracy: 0.2983, AUC: 0.5097
Epoch 3, Loss: 1.8604, Test Loss: 1.8238, Accuracy: 0.3496, AUC: 0.5195
Epoch 4, Loss: 1.8234, Test Loss: 1.7878, Accuracy: 0.3862, AUC: 0.5291
Epoch 5, Loss: 1.7875, Test Loss: 1.7529, Accuracy: 0.4119, AUC: 0.5392
Epoch 6, Loss: 1.7526, Test Loss: 1.7190, Accuracy: 0.4311, AUC: 0.5504
Epoch 7, Loss: 1.7187, Test Loss: 1.6861, Accuracy: 0.4461, AUC: 0.5621
Epoch 8, Loss: 1.6857, Test Loss: 1.6541, Accuracy: 0.4562, AUC: 0.5733
Epoch 9, Loss: 1.6537, Test Loss: 1.6230, Accuracy: 0.4662, AUC: 0.5838
Epoch 10, Loss: 1.6227, Test Loss: 1.5929, Accuracy: 0.4757, AUC: 0.5941
Epoch 11, Loss: 1.5925, Test Loss: 1.5636, Accuracy: 0.4866, AUC: 0.6043
Epoch 12, Loss: 1.5632, Test Loss: 1.5351, Accuracy: 0.4987, AUC: 0.6142
Epoch 13, Loss: 1.5347, Test Loss: 1.5074, Accuracy: 0.5100, AUC: 0.6236
Epoch 14, Loss: 1.5069, Test Loss: 1.4805, Accuracy: 0.5200,